In [ ]:
import xarray as xr
import pystac_client
import planetary_computer
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

from dask.distributed import Client, LocalCluster

In [ ]:
# TODO: Create a `LocalCluster` object with `n_workers` set to 4 
#       and `threads_per_worker` set to 4
cluster = LocalCluster(n_workers=4, threads_per_worker=4)

# TODO: Create a `Client` object that takes the `cluster` 
#       object from the prior step as input
client = Client(cluster)
client

In [ ]:
# TODO: Set the Tucson latitude and longitude. 
#       Make sure to use the longitude on the 
#       0 to 360 interval where 0 is the prime meridian
tucson_lat = 32.2226
tucson_lon = 360 - 110.9747
tucson_lon

In [ ]:
path = 'gs://gcp-public-data-arco-era5/ar/1959-2022-6h-1440x721.zarr'
era5_ds = xr.open_dataset(path, engine='zarr', chunks='auto')
era5_temperature = era5_ds['2m_temperature']

In [ ]:
#TODO: Create a `slice` object that selects every 7th point. Reminder slice notation has 
# the format of `start, stop, step`.
era5_time_slice = slice(0, None, 7)

#TODO: Now perform the following operations:
# 1. Select the location for the `tucson_lat` and `tucson_lon` from `era5_temperature`
#    Make sure to use the `method='nearest'` argument to make sure you will find a location.
# 2. Use the `era5_time_slice` to "index" select the timestamps
# 3. `Resample` the dataset to monthly (using the '1M' string as the value supplied to the `time` key)
# 4. Run the `.mean()` function on the resulting data. 
# 5. Finally, make sure to `.compute()` the values and assign the result to 
#    the variable `tucson_era5_temperature`. This step may take a couple of minutes.
tucson_era5_temperature = era5_temperature.sel(latitude=tucson_lat, longitude=tucson_lon, method='nearest')
tucson_era5_temperature = tucson_era5_temperature.isel(time=era5_time_slice)
tucson_era5_temperature = tucson_era5_temperature.resample(time='1M').mean().compute()

In [ ]:
# Nothing to do here
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1/",
    modifier=planetary_computer.sign_inplace,
)
collection = catalog.get_collection("cil-gdpcir-cc-by")
summary = collection.summaries.to_dict()
available_keys = list(summary.keys())
print(available_keys)

In [ ]:
# Nothing to do here
# These are the variables available
summary['cmip6:variable']

In [ ]:
# Nothing to do here
# These are the models available
summary['cmip6:source_id']

In [ ]:
# Nothing to do here
# These are the scenarios available
summary['cmip6:experiment_id']

In [ ]:
# TODO: Fill these out, first, choose any `model`` you would like, 
#       as well as the SSP2-4.5 `scenario`.
model = None
scenario = None

# Nothing to do here
search_245 = catalog.search(
    collections=['cil-gdpcir-cc-by'],
    query={
        'cmip6:source_id': {'eq': model},
        'cmip6:experiment_id': {'eq': scenario}
    }
)
cmip_245_data = search_245.item_collection()

In [ ]:
print(tucson_lon-360)

In [ ]:
# TODO: Fill these out. You may want to show the result of `gfdl_245_data` 
#       to find these under the `assets` dropdown.
tmax_varname = None
tmin_varname = None

# Nothing needed here
cmip_245_tmax = cmip_245_data[0].assets[tmax_varname]
cmip_245_tmax = xr.open_dataset(
    cmip_245_tmax.href,
    **cmip_245_tmax.extra_fields["xarray:open_kwargs"]
).chunk('auto')

# Nothing needed here
cmip_245_tmin = cmip_245_data[0].assets[tmin_varname]
cmip_245_tmin = xr.open_dataset(
    cmip_245_tmin.href,
    **cmip_245_tmin.extra_fields["xarray:open_kwargs"]
).chunk('auto')

# TODO: Use the xr.merge command to merge these together
cmip_245_ds = None

# TODO: Now calculate `tavg` as `(3 * tmax + tmin) / 4`
cmip_245_ds['tavg'] = None

In [ ]:
# TODO: Create a `slice` object that selects every 3rd point. Reminder slice notation has 
# the format of `start, stop, step`.
cmip_time_slice = None

# Nothing to do here. This is the same operation you did for the era5 data.
# Note: This step will take a while - probably ~10 mins
# Note: We subtract 360 from the lon because the CIL data uses a (-180, 180) 
tucson_cmip_245_temp = cmip_245_ds['tavg'].sel(
    lat=tucson_lat, lon=tucson_lon-360, method='nearest'
).isel(time=cmip_time_slice).resample(time='1M').mean().compute()

# Nothing to do here. This just fixes some timestamps
tucson_cmip_245_temp['time'] = tucson_cmip_245_temp['time'].astype('datetime64[ns]')

In [ ]:
# TODO: Now repeat the last 3 code cells, but select out the SSP5-8.5 scenario
tucson_cmip_585_temp = None

In [ ]:
# TODO: Now repeat the last 3 code cells, but select out the SSP5-8.5 scenario
tucson_cmip_hist_temp = None

In [ ]:
# Nothing for you to do directly here
tucson_era5_temperature.groupby('time.year').mean().plot(color='black', label='historic')
tucson_cmip_hist_temp.groupby('time.year').mean().plot(color='darkgrey', linestyle='--', label='model')
tucson_cmip_245_temp.groupby('time.year').mean().plot(color='steelblue', label='SSP2-4.5')
tucson_cmip_585_temp.groupby('time.year').mean().plot(color='tomato', label='SSP5-8.5')
plt.legend()

In [ ]:
# TODO: Write up some reflection from this exercise. 
#  - What were we analyzing and why? 
#  - Are the results as you expect? 
#  - How might you extend the analysis to be more actionable?

In [ ]:
# TODO: For 501 students only (2pts):
# Align the historic data so that there are equal timestamps between the two
# Fit a linear regression between the era5 and historic data 
# Then, apply it to the SSP2-4.5 and SSP5-8.5 projections 
# Finally, make the same plot as above for the "corrected" data